<a href="https://colab.research.google.com/github/Parv-065040/MLM_Proj1/blob/main/mlm25_proj1_024040060.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

google_sheets_url = 'https://docs.google.com/spreadsheets/d/1z9zRFXOfcZKovjIS9j-ZVnVoyK3X89sQRv03CP8a07g/gviz/tq?tvd=json&tqx=out:csv&gid=2052849645'
df = pd.read_csv(google_sheets_url)
print("Dataset loaded successfully. First 5 rows:")
print(df.head())

Dataset loaded successfully. First 5 rows:
    Product Name   Category  Daily Production/Quantity Transportation Method  \
0  Cluster Beans  Vegetable                       1342               By Road   
1   Dry Chillies      Spice                      23934               By Road   
2   Elephant Yam  Root Crop                      14384               By Road   
3      Drumstick  Vegetable                      15799               By Road   
4         Banana      Fruit                       1761               By Road   

            Vehicle Type  Vehicle Capacity (kg/ton)  \
0             Mini Truck                        1.0   
1             Mini Truck                        1.5   
2             Bulk Truck                       28.0   
3  Mini Refrigerated Van                        1.0   
4  Mini Refrigerated Van                        1.0   

   No. of Vehicles Used Per Day  Trip Frequency  \
0                             1               1   
1                             4            

In [ ]:
df.shape

(100000, 32)

In [ ]:
df.shape
df.info
df.describe(include='all')

,Product Name,Category,Daily Production/Quantity,Transportation Method,Vehicle Type,Vehicle Capacity (kg/ton),No. of Vehicles Used Per Day,Trip Frequency,Average Distance Travelled (km),Temperature Maintained (°C),...,Is Real-time Tracking Used?,Tracking Method,Data Collected,Initial Quality Grade,Shelf Life (Days),Spoilage Percentage (%),Reasons for Spoilage,Transportation Cost per Trip,Storage Cost,Wastage Cost
count,100000,100000,100000.000000,100000,100000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,...,100000,50125,100000,100000,100000.000000,100000.000000,100000,100000.000000,100000.000000,100000.000000
unique,33,7,NaN,1,7,NaN,NaN,NaN,NaN,NaN,...,2,4,3,5,NaN,NaN,8,NaN,NaN,NaN
top,Water Melon,Vegetable,NaN,By Road,Mini Refrigerated Van,NaN,NaN,NaN,NaN,NaN,...,Yes,RFID + GPS,Quantity,B+,NaN,NaN,Delay in transit,NaN,NaN,NaN
freq,3109,36376,NaN,100000,14446,NaN,NaN,NaN,NaN,NaN,...,50125,12599,49875,20079,NaN,NaN,12689,NaN,NaN,NaN
mean,NaN,NaN,12901.294000,NaN,NaN,8.003605,2.992370,1.999710,370.579140,15.051950,...,NaN,NaN,NaN,NaN,63.316830,3.315252,NaN,15924.755130,6348.269430,2105.968750
std,NaN,NaN,6994.142258,NaN,NaN,6.956875,1.413326,0.817225,191.125051,8.816203,...,NaN,NaN,NaN,NaN,95.057091,2.208593,NaN,8153.812284,3253.841849,1095.585359
min,NaN,NaN,800.000000,NaN,NaN,1.000000,1.000000,1.000000,40.000000,2.000000,...,NaN,NaN,NaN,NaN,2.000000,0.500000,NaN,1800.000000,700.000000,200.000000
25%,NaN,NaN,6840.000000,NaN,NaN,3.000000,2.000000,1.000000,206.000000,6.000000,...,NaN,NaN,NaN,NaN,11.000000,1.600000,NaN,8863.000000,3528.000000,1158.000000
50%,NaN,NaN,12910.000000,NaN,NaN,6.000000,3.000000,2.000000,371.000000,12.000000,...,NaN,NaN,NaN,NaN,20.000000,2.800000,NaN,15977.000000,6354.000000,2109.000000
75%,NaN,NaN,18942.000000,NaN,NaN,10.000000,4.000000,3.000000,536.000000,23.000000,...,NaN,NaN,NaN,NaN,30.000000,4.400000,NaN,22980.000000,9158.000000,3052.000000


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, DBSCAN, SpectralClustering
from sklearn.metrics import silhouette_score, davies_bouldin_score


In [ ]:
# Ditsya Banerjee (065024)
db024_df = df.sample(n=5001, random_state=24).reset_index(drop=True)

# Parv (065040)
pp040_df = df.sample(n=5001, random_state=40).reset_index(drop=True)

# Awantika Kholia (065060)
ak060_df = df.sample(n=5001, random_state=60).reset_index(drop=True)


In [ ]:
db024_pp040_ak060_df = pd.concat(
    [db024_df, pp040_df, ak060_df],
    axis=0
).reset_index(drop=True)

db024_pp040_ak060_df.shape


(15003, 32)

In [ ]:
def clean_data_unsup(data):
    # Remove columns with ≥50% missing
    data = data.loc[:, data.isnull().mean() < 0.5]

    # Remove rows with ≥50% missing
    data = data.loc[data.isnull().mean(axis=1) < 0.5]

    # Impute remaining missing values
    for col in data.columns:
        if data[col].isnull().sum() > 0:
            if data[col].dtype == 'object':
                data[col].fillna(data[col].mode()[0], inplace=True)
            else:
                data[col].fillna(data[col].median(), inplace=True)
    return data


In [ ]:
db024_df = clean_data_unsup(db024_df)


/tmp/ipython-input-4249638738.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[col].fillna(data[col].mode()[0], inplace=True)


In [ ]:
import plotly.express as px

category_counts_df = df['Category'].value_counts().reset_index()
category_counts_df.columns = ['Category', 'count']

fig = px.bar(
    category_counts_df,
    x='Category', y='count',
    color='Category',
    title="Category Distribution (Interactive)",
    labels={'Category': 'Category', 'count': 'Count'},
)
fig.show()

In [ ]:
fig = px.violin(
    df,
    y="Transportation Cost per Trip",
    box=True, points="all",
    color_discrete_sequence=['#ff7f0e'],
    title="Transportation Cost Distribution (Violin Plot)"
)
fig.show()


In [ ]:
fig = px.scatter(
    df,
    x="Average Distance Travelled (km)",
    y="Transportation Cost per Trip",
    color="Category",
    title="Distance vs Transportation Cost (Interactive Scatter)",
    template="plotly_dark"
)
fig.show()


In [ ]:
import plotly.express as px

correlation_matrix = df.select_dtypes(exclude='object').corr()

fig = px.imshow(
    correlation_matrix,
    text_auto=True, # Show correlation values on heatmap
    aspect="auto",
    color_continuous_scale='RdBu_r', # Red-blue diverging colormap, reversed
    title='Correlation Heatmap of Numerical Variables (Interactive & Exotic)',
    labels=dict(x="Features", y="Features", color="Correlation"),
    x=correlation_matrix.columns,
    y=correlation_matrix.columns
)
fig.update_layout(
    template='plotly_dark',
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    font=dict(size=10)
)
fig.show()

In [ ]:
X_db = db024_df.copy()

cat_cols = X_db.select_dtypes(include='object').columns
num_cols = X_db.select_dtypes(exclude='object').columns

preprocessor_db = ColumnTransformer([
    ('num', StandardScaler(), num_cols),
    ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), cat_cols)
])

X_db_scaled = preprocessor_db.fit_transform(X_db)


In [ ]:
pca_db = PCA(n_components=0.85, random_state=24, svd_solver='full')
X_db_pca = pca_db.fit_transform(X_db_scaled.toarray())

In [ ]:
import plotly.graph_objects as go

explained_variance_ratio_cumsum = np.cumsum(pca_db.explained_variance_ratio_)

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=np.arange(1, len(explained_variance_ratio_cumsum) + 1),
    y=explained_variance_ratio_cumsum,
    mode='lines+markers',
    name='Cumulative Explained Variance'
))
fig.add_hline(y=0.85, line_dash="dot", line_color="red", annotation_text="85% Variance", annotation_position="bottom right")

fig.update_layout(
    title='PCA Explained Variance',
    xaxis_title='Number of Components',
    yaxis_title='Cumulative Explained Variance',
    template='plotly_dark'
)
fig.show()

In [ ]:
import plotly.graph_objects as go

inertia = []
for k in range(2, 9):
    km = KMeans(n_clusters=k, random_state=24, n_init='auto')
    km.fit(X_db_pca)
    inertia.append(km.inertia_)

fig = go.Figure(data=go.Scatter(x=list(range(2, 9)), y=inertia, mode='lines+markers'))
fig.update_layout(
    title='Elbow Method – KMeans',
    xaxis_title='K',
    yaxis_title='Inertia',
    template='plotly_dark'
)
fig.show()

In [ ]:
db024_kmeans = KMeans(n_clusters=4, random_state=24)
db024_labels = db024_kmeans.fit_predict(X_db_pca)

print("Silhouette Score:",
      silhouette_score(X_db_pca, db024_labels))
print("Davies-Bouldin Index:",
      davies_bouldin_score(X_db_pca, db024_labels))


Silhouette Score: 0.06709614979543903
Davies-Bouldin Index: 2.9401941795695414


In [ ]:
import plotly.express as px

# Create a DataFrame for plotting with PCA components and KMeans labels
df_pca_clustered = pd.DataFrame(X_db_pca, columns=[f'PC{i+1}' for i in range(X_db_pca.shape[1])])
df_pca_clustered['KMeans_Cluster'] = db024_labels

fig = px.scatter(
    df_pca_clustered,
    x='PC1',
    y='PC2',
    color='KMeans_Cluster',
    title='KMeans Clusters',
    hover_data={'PC1': ':.2f', 'PC2': ':.2f', 'KMeans_Cluster': True}
)
fig.update_layout(
    xaxis_title="PC1",
    yaxis_title="PC2",
    template='plotly_dark'
)
fig.show()

In [ ]:
pp040_df = clean_data_unsup(pp040_df)


In [ ]:
X_pp = pp040_df.copy()

preprocessor_pp = ColumnTransformer([
    ('num', StandardScaler(), X_pp.select_dtypes(exclude='object').columns),
    ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'),
     X_pp.select_dtypes(include='object').columns)
])

X_pp_scaled = preprocessor_pp.fit_transform(X_pp)

pca_pp = PCA(n_components=0.85, random_state=40, svd_solver='full')
X_pp_pca = pca_pp.fit_transform(X_pp_scaled.toarray())

In [ ]:
pp040_dbscan = DBSCAN(eps=1.5, min_samples=10)
pp040_labels = pp040_dbscan.fit_predict(X_pp_pca)


In [ ]:
mask = pp040_labels != -1

# Check if there are any non-noise points before calculating silhouette score
if np.any(mask):
    # Further check if there's more than one unique label among the non-noise points
    # Silhouette score requires at least 2 clusters (excluding noise)
    unique_labels_in_mask = np.unique(pp040_labels[mask])
    if len(unique_labels_in_mask) > 1:
        print("Silhouette Score (excluding noise):",
              silhouette_score(X_pp_pca[mask], pp040_labels[mask]))
    else:
        print("DBSCAN found only one cluster (or fewer than 2 distinct clusters) after removing noise points. Silhouette score requires at least two distinct clusters to be computed.")
        print("Consider adjusting 'eps' and 'min_samples' parameters in DBSCAN to form more clusters.")
else:
    print("DBSCAN found no non-noise points (all points are noise). Cannot compute Silhouette Score.")
    print("Consider adjusting 'eps' and 'min_samples' parameters in DBSCAN to form clusters.")

DBSCAN found no non-noise points (all points are noise). Cannot compute Silhouette Score.
Consider adjusting 'eps' and 'min_samples' parameters in DBSCAN to form clusters.


In [ ]:
import plotly.express as px

# Create a DataFrame for plotting with PCA components and DBSCAN labels
df_dbscan_clustered = pd.DataFrame(X_pp_pca, columns=[f'PC{i+1}' for i in range(X_pp_pca.shape[1])])
df_dbscan_clustered['DBSCAN_Cluster'] = pp040_labels

fig = px.scatter(
    df_dbscan_clustered,
    x='PC1',
    y='PC2',
    color='DBSCAN_Cluster',
    title='DBSCAN Clusters -1 = Noise)',
    hover_data={'PC1': ':.2f', 'PC2': ':.2f', 'DBSCAN_Cluster': True}
)
fig.update_layout(
    xaxis_title="PC1",
    yaxis_title="PC2",
    template='plotly_dark'
)
fig.show()

In [ ]:
ak060_df = clean_data_unsup(ak060_df)


/tmp/ipython-input-4249638738.py:12: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.





In [ ]:
X_ak = ak060_df.copy()

preprocessor_ak = ColumnTransformer([
    ('num', StandardScaler(), X_ak.select_dtypes(exclude='object').columns),
    ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'),
     X_ak.select_dtypes(include='object').columns)
])

X_ak_scaled = preprocessor_ak.fit_transform(X_ak)

pca_ak = PCA(n_components=0.85, random_state=60, svd_solver='full')
X_ak_pca = pca_ak.fit_transform(X_ak_scaled.toarray())

In [ ]:
ak060_spectral = SpectralClustering(
    n_clusters=4,
    affinity='nearest_neighbors',
    random_state=60
)

ak060_labels = ak060_spectral.fit_predict(X_ak_pca)


In [ ]:
print("Silhouette Score:",
      silhouette_score(X_ak_pca, ak060_labels))
print("Davies-Bouldin Index:",
      davies_bouldin_score(X_ak_pca, ak060_labels))


Silhouette Score: 0.0627459307824285
Davies-Bouldin Index: 3.018984960122527


In [ ]:
import plotly.express as px

# Create a DataFrame for plotting with PCA components and Spectral Clustering labels
df_spectral_clustered = pd.DataFrame(X_ak_pca, columns=[f'PC{i+1}' for i in range(X_ak_pca.shape[1])])
df_spectral_clustered['Spectral_Cluster'] = ak060_labels

fig = px.scatter(
    df_spectral_clustered,
    x='PC1',
    y='PC2',
    color='Spectral_Cluster',
    title='Spectral Clustering',
    hover_data={'PC1': ':.2f', 'PC2': ':.2f', 'Spectral_Cluster': True}
)
fig.update_layout(
    xaxis_title="PC1",
    yaxis_title="PC2",
    template='plotly_dark'
)
fig.show()

In [ ]:
db024_pp040_ak060_df = clean_data_unsup(db024_pp040_ak060_df)


/tmp/ipython-input-4249638738.py:12: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.





In [ ]:
preprocessor_g = ColumnTransformer([
    ('num', StandardScaler(), db024_pp040_ak060_df.select_dtypes(exclude='object').columns),
    ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'),
     db024_pp040_ak060_df.select_dtypes(include='object').columns)
])

Xg_scaled = preprocessor_g.fit_transform(db024_pp040_ak060_df)

pca_g = PCA(n_components=0.85, svd_solver='full')
Xg_pca = pca_g.fit_transform(Xg_scaled.toarray())

In [ ]:
group_kmeans = KMeans(n_clusters=4, random_state=24)
group_labels = group_kmeans.fit_predict(Xg_pca)

db024_pp040_ak060_df['Cluster'] = group_labels


In [ ]:
db024_pp040_ak060_df.groupby('Cluster').mean(numeric_only=True)


,Daily Production/Quantity,Vehicle Capacity (kg/ton),No. of Vehicles Used Per Day,Trip Frequency,Average Distance Travelled (km),Temperature Maintained (°C),Humidity Maintained (%),Transport Duration (hours),Unloading Time,Total Supply Chain Time,Daily Demand,Shelf Life (Days),Spoilage Percentage (%),Transportation Cost per Trip,Storage Cost,Wastage Cost
Cluster,,,,,,,,,,,,,,,,
0,12921.507522,8.048895,2.960743,1.989187,371.567466,11.889986,82.958862,14.606088,1.764833,20.265891,12905.948519,16.115421,3.940762,15503.239774,6326.750588,2093.571462
1,6453.583448,7.703310,3.044966,2.002483,370.392552,11.756414,82.857103,6.336028,1.599945,11.659697,6434.152828,16.040828,3.974841,15709.087172,6297.317793,2091.765793
2,13026.360091,7.993880,2.994876,2.020211,370.084543,25.038998,54.805295,9.587475,1.632195,14.928039,13040.374039,214.715628,1.243866,15886.031882,6199.392542,2066.958725
3,19455.979784,8.530878,2.953199,1.984215,368.009693,12.364165,82.816117,6.418194,1.600720,11.674550,19488.444475,16.704514,3.922071,16495.960399,6399.662697,2172.027970


In [ ]:
import plotly.express as px

fig = px.box(
    db024_pp040_ak060_df,
    x='Cluster',
    y='Transportation Cost per Trip',
    title='Cluster-wise Transportation Cost',
    color='Cluster', # Differentiate boxes by cluster
    hover_data={'Cluster': True, 'Transportation Cost per Trip': ':.2f'}
)
fig.update_layout(
    xaxis_title="Cluster",
    yaxis_title="Transportation Cost per Trip",
    template='plotly_dark'
)
fig.show()

In [ ]:
import pandas as pd

kmeans_metrics = {
    'Silhouette Score': 0.06709614979543903,
    'Davies-Bouldin Index': 2.9401941795695414
}

spectral_metrics = {
    'Silhouette Score': 0.0627459307824285,
    'Davies-Bouldin Index': 3.018984960122527
}

comparison_df = pd.DataFrame({
    'KMeans': kmeans_metrics,
    'Spectral Clustering': spectral_metrics
}).T

comparison_df.index.name = 'Clustering Algorithm'
print("Comparison of Clustering Metrics:")
print(comparison_df)

Comparison of Clustering Metrics:
                      Silhouette Score  Davies-Bouldin Index
Clustering Algorithm                                        
KMeans                        0.067096              2.940194
Spectral Clustering           0.062746              3.018985


In [ ]:
import plotly.graph_objects as go

# Bar chart for Silhouette Score
fig_silhouette = go.Figure(
    data=[
        go.Bar(
            name='Silhouette Score',
            x=comparison_df.index,
            y=comparison_df['Silhouette Score']
        )
    ]
)
fig_silhouette.update_layout(
    title='Comparison of Silhouette Scores',
    xaxis_title='Clustering Algorithm',
    yaxis_title='Silhouette Score',
    template='plotly_dark'
)
fig_silhouette.show()

# Bar chart for Davies-Bouldin Index
fig_davies_bouldin = go.Figure(
    data=[
        go.Bar(
            name='Davies-Bouldin Index',
            x=comparison_df.index,
            y=comparison_df['Davies-Bouldin Index']
        )
    ]
)
fig_davies_bouldin.update_layout(
    title='Comparison of Davies-Bouldin Indices',
    xaxis_title='Clustering Algorithm',
    yaxis_title='Davies-Bouldin Index',
    template='plotly_dark'
)
fig_davies_bouldin.show()